In [1]:
import pickle
import numpy as np
import math

# please do not re-generate the file 
- from sklearn.cluster import KMeans
- generated by KMeans(n_clusters=1000,init='random', random_state=10).fit(train_data['data'])

In [2]:
train_data=pickle.load(open( "train_PCA.pkl", "rb" ))
test_data=pickle.load(open( "test_PCA.pkl", "rb" ))

train_cluster=pickle.load(open( "Kmean_cluster.pkl", "rb" ))
print(train_cluster)
print(train_data['target'].shape)

[955 182 293 ...,  35 974 881]
(2034,)


In [3]:
mu = np.zeros([1000,1500], dtype=float)
data_by_class = [[] for i in range(1000)]

for i, vector in enumerate(train_data['data']):
    label = train_cluster[i]
    data_by_class[label].append(vector)
    
for i, vector_list in enumerate(data_by_class):
    mu[i] = np.mean(vector_list,axis=0)
#     print(mu[i],len(vector_list))
# print (mu)
print(np.linalg.det(train_data['data'].T.dot(train_data['data'])))

0.0102791969603


In [4]:
Sigma_s = np.zeros([1000], dtype=float)
for i in range(1000):
    min_value = -1
    for vector in train_data['data']:
        dis = np.sum(np.square(vector - mu[i]))
        if dis <= 0.0:
            continue
        if min_value == -1:
            min_value = dis
        elif min_value > dis:
            min_value = dis
    Sigma_s[i] = min_value
print(Sigma_s)

[  1.99813803e-01   9.39566819e-01   1.27130009e+00   8.70615048e-01
   4.50606559e-01   1.63212501e-01   3.09312398e-01   2.68227948e-01
   7.48550943e-01   5.87178238e-01   1.66119130e+00   3.56291000e-01
   1.45413372e-01   1.61987944e+00   1.41855588e-01   1.56655828e+00
   3.42932528e-01   1.38931620e+00   3.41301556e-01   1.54809733e+00
   2.62553716e-01   1.11462170e-01   2.47004420e-01   1.69266007e+00
   5.59853053e-01   3.21589108e-01   5.23684163e-02   7.68063298e-01
   9.86177882e-02   1.68392268e+00   3.09742713e-01   1.33599979e+00
   2.12612518e-01   8.94411572e-02   3.09242679e-01   4.83018900e-01
   7.61645696e-02   2.58140010e-01   1.31097491e-01   1.51799559e-01
   4.35132695e-01   3.63698103e-01   1.56546698e-01   4.34927932e-01
   3.96562806e-01   1.36938792e-04   3.78376192e-01   1.68920307e+00
   9.36182769e-02   5.04747548e-01   3.55146307e-01   1.69876672e-01
   1.46675610e+00   6.12010684e-02   2.10804561e-01   1.59003842e+00
   1.70033592e+00   2.11097305e-01

In [5]:
shrunk_data = np.zeros([2034,1000], dtype=float)
for i in range(2034):
    vector = train_data['data'][i]
# for i, vector in enumerate(train_data['data']):
#     print(vector.shape, mu[i].shape)
#     print(sum(np.square(vector-mu[i]).shape))
    for j in range(1000):
#         print(-0.5*sum(np.square(vector-mu[j]))/np.square(Sigma[j]))
        temp = vector - mu[j]
        temp = -np.dot(temp,temp.T)/(2*Sigma_s[j])
        temp = math.exp(temp)
        shrunk_data[i,j] = temp
#         shrunk_data[i,j] = np.exp(-0.5*np.sum(np.square(vector-mu[j]))/Sigma_s[j])
#     print shrunk_data[i]

In [6]:
print(np.linalg.det(shrunk_data.T.dot(shrunk_data)))

0.0


In [7]:
save_data = dict()
print(train_data['target'].shape)
save_data['target'] = train_data['target']
save_data['data'] = shrunk_data

f = open( "train_PCA_sk.pkl", "wb" )
train_data=pickle.dump(save_data, f)
f.close()

(2034,)


In [8]:
test_data['data'].shape

(1353, 1500)

In [9]:
shrunk_data = np.zeros([1353,1000], dtype=float)
for i, vector in enumerate(test_data['data']):
    for j in range(1000):
#         shrunk_data[i,j] = np.exp(-0.5*sum(np.square(vector-mu[j]))/np.square(Sigma[j]))
        temp = vector - mu[j]
        temp = -np.dot(temp,temp.T)*1.0/(2*Sigma_s[j])
        temp = math.exp(temp)
        shrunk_data[i,j] = temp

In [10]:
save_data = dict()
save_data['target'] = test_data['target']
save_data['data'] = shrunk_data
f = open( "test_PCA_sk.pkl", "wb" )
train_data=pickle.dump(save_data, f)
f.close()